# Testing file 
### where we evaluate LAFTR's models using the test set

## Preliminaries

In [1]:
import pandas as pd
from tensorflow.data import Dataset
from tensorflow.keras.optimizers import Adam


from util.load_data import load_data
from util.evaluation import *
from models.madras_laftr.models import *
from models.madras_laftr.learning import train_loop as laftr_train

In [2]:
batch_size = 64
epochs = 1
learning_rate = 0.001
opt = Adam(learning_rate=learning_rate)

In [3]:
CLAS_COEFF = 1.
FAIR_COEFFS = [.2, .5, .7, 1.]
RECON_COEFF = 0.
hidden_layer_specs = {'clas':[8] , 'enc':[8] , 'dec':[8] , 'adv':[8]}

In [4]:
test_loop = 1

## Load data

In [5]:
data_name = 'adult'

In [6]:
x_train, y_train, a_train = load_data(data_name, 'train')
raw_data = (x_train, y_train, a_train)

In [7]:
xdim = x_train.shape[1]
ydim = y_train.shape[1]
adim = a_train.shape[1]
zdim = 8

In [8]:
train_data = Dataset.from_tensor_slices((x_train, y_train, a_train))
train_data = train_data.batch(batch_size, drop_remainder=True)
train_data

<BatchDataset shapes: ((64, 112), (64, 1), (64, 1)), types: (tf.float64, tf.float64, tf.float64)>

In [9]:
x_valid, y_valid, a_valid = load_data(data_name, 'valid')

valid_data = Dataset.from_tensor_slices((x_valid, y_valid, a_valid))
valid_data = valid_data.batch(batch_size, drop_remainder=True)

In [10]:
x_test, y_test, a_test = load_data(data_name, 'test')

test_data = Dataset.from_tensor_slices((x_test, y_test, a_test))
test_data = test_data.batch(batch_size, drop_remainder=True)

## Result file

In [11]:
header = "model_name", "fair_coeff", "clas_acc", "dp", "deqodds", "deqopp", "trade_dp", "trade_deqodds", "trade_deqopp", "TN_a0", "FP_a0", "FN_a0", "TP_a0", "TN_a1", "FP_a1", "FN_a1", "TP_a1"
results = []

## Testing loop
#### Each model is evalueted 5 times
#### In the end of each iteration we save the result

### LAFTR for DP

In [14]:
for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)

        model = DemParGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4DP', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.1448388695716858 | 0.5440177917480469 | 0.5475058555603027 | 0.9201741218566895 | 0.6901939655172413 | 0.6080487400530504 | -101.54302055702918
> Class Acc | Adv Acc
> 0.7779255319148937 | 0.6737367021276596
> DP | DEqOdds | DEqOpp
> 0.9494310729205608 | 0.9315301668830216 | 0.8760752081871033
> Confusion Matrix 
TN: 11251.0 | FP: 91.0 
FN: 3249.0 | TP: 449.0
> Confusion Matrix for A = 0 
TN: 4350.0 | FP: 0.0 
FN: 548.0 | TP: 9.0
> Confusion Matrix for A = 1 
TN: 6901.0 | FP: 91.0 
FN: 2701.0 | TP: 440.0
> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.0933845043182373 | 0.5472743511199951 | 0.5342416763305664 | 0.9201741218566895 | 0.6905255305039788 | 0.5975629973474801 | -101.54302055702918
> Class Acc | Adv Acc
> 0.7801196808510639 | 0.6737367021276596
> DP | DEqOdds | DEqOpp
> 0.9441083669662476 | 0.9236951800994575 | 0.8617485761642456
> C

### LAFTR for Eq Odds

In [16]:
for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)

        model = EqOddsUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOdds', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.015396754257380962 | 0.5530157089233398 | 0.5068254470825195 | 0.9201741218566895 | 0.6868368700265252 | 0.622886273209549 | -101.54302055702918
> Class Acc | Adv Acc
> 0.76875 | 0.6737367021276596
> DP | DEqOdds | DEqOpp
> 0.970352815464139 | 0.961163013940677 | 0.9296762347221375
> Confusion Matrix 
TN: 11288.0 | FP: 54.0 
FN: 3424.0 | TP: 274.0
> Confusion Matrix for A = 0 
TN: 4349.0 | FP: 1.0 
FN: 549.0 | TP: 8.0
> Confusion Matrix for A = 1 
TN: 6939.0 | FP: 53.0 
FN: 2875.0 | TP: 266.0
> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.015396754257380962 | 0.5530157089233398 | 0.5068254470825195 | 0.9201741218566895 | 0.6868368700265252 | 0.622886273209549 | -101.54302055702918
> Class Acc | Adv Acc
> 0.76875 | 0.6737367021276596
> DP | DEqOdds | DEqOpp
> 0.970352815464139 | 0.961163013940677 | 0.9296762347221375
> Confusion Matrix 
TN: 112

### LAFTR for Eq Opp

In [17]:
for FAIR_COEFF in FAIR_COEFFS:
    for i in range(test_loop):

        opt = Adam(learning_rate=learning_rate)

        model = EqOppUnweightedGan(xdim, ydim, adim, zdim, hidden_layer_specs, recon_coeff=RECON_COEFF, clas_coeff=CLAS_COEFF, fair_coeff=FAIR_COEFF)
        ret = laftr_train(model, raw_data, train_data, epochs, opt)

        Y, A, Y_hat, A_hat = fair_evaluation(model, valid_data)
        clas_acc, dp, deqodds, deqopp, confusion_matrix, metrics_a0, metrics_a1 = compute_metrics(Y, A, Y_hat, A_hat, adim)

        fair_metrics = (dp, deqodds, deqopp)
        tradeoff = []
        for fair_metric in fair_metrics:
            tradeoff.append(compute_tradeoff(clas_acc, fair_metric))

        result = ['LAFTR4EqOpp', FAIR_COEFF, clas_acc, dp, deqodds, deqopp, tradeoff[0], tradeoff[1], tradeoff[2]] + metrics_a0 + metrics_a1

        results.append(result)

> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.0058588190004229546 | 0.5432302355766296 | 0.5288085341453552 | 0.9201741218566895 | 0.6884946949602122 | 0.5864555702917772 | -101.54302055702918
> Class Acc | Adv Acc
> 0.7837101063829788 | 0.6737367021276596
> DP | DEqOdds | DEqOpp
> 0.9336988180875778 | 0.9130467846989632 | 0.8451407551765442
> Confusion Matrix 
TN: 11201.0 | FP: 141.0 
FN: 3112.0 | TP: 586.0
> Confusion Matrix for A = 0 
TN: 4347.0 | FP: 3.0 
FN: 542.0 | TP: 15.0
> Confusion Matrix for A = 1 
TN: 6854.0 | FP: 138.0 
FN: 2570.0 | TP: 571.0
> Epoch | Model Loss | Class Loss | Adv Loss | Dec Loss | Class Acc | Adv Acc | Dec Acc
> 1 | 0.0058588190004229546 | 0.5432302355766296 | 0.5288085341453552 | 0.9201741218566895 | 0.6884946949602122 | 0.5864555702917772 | -101.54302055702918
> Class Acc | Adv Acc
> 0.7837101063829788 | 0.6737367021276596
> DP | DEqOdds | DEqOpp
> 0.9336988180875778 | 0.9130467846989632 | 0.84514075517

## Saving into DF then CSV

In [18]:
result_df = pd.DataFrame(results, columns=header)
result_df

,model_name,fair_coeff,clas_acc,dp,deqodds,deqopp,trade_dp,trade_deqodds,trade_deqopp,TN_a0,FP_a0,FN_a0,TP_a0,TN_a1,FP_a1,FN_a1,TP_a1
0,LAFTR4DP,0.2,0.777926,0.949431,0.931530,0.876075,0.855164,0.847827,0.824088,4350.0,0.0,548.0,9.0,6901.0,91.0,2701.0,440.0
1,LAFTR4DP,0.5,0.780120,0.944108,0.923695,0.861749,0.854316,0.845858,0.818905,4349.0,1.0,548.0,9.0,6890.0,102.0,2656.0,485.0
2,LAFTR4DP,0.7,0.776596,0.951602,0.934074,0.880647,0.855238,0.848086,0.825355,4349.0,1.0,549.0,8.0,6903.0,89.0,2721.0,420.0
3,LAFTR4DP,1.0,0.767553,0.973313,0.965413,0.937317,0.858273,0.855188,0.843983,4349.0,1.0,549.0,8.0,6945.0,47.0,2899.0,242.0
4,LAFTR4EqOdds,0.2,0.768750,0.970353,0.961163,0.929676,0.857866,0.854256,0.841589,4349.0,1.0,549.0,8.0,6939.0,53.0,2875.0,266.0
5,LAFTR4EqOdds,0.5,0.768750,0.970353,0.961163,0.929676,0.857866,0.854256,0.841589,4349.0,1.0,549.0,8.0,6939.0,53.0,2875.0,266.0
6,LAFTR4EqOdds,0.7,0.768750,0.970353,0.961163,0.929676,0.857866,0.854256,0.841589,4349.0,1.0,549.0,8.0,6939.0,53.0,2875.0,266.0
7,LAFTR4EqOdds,1.0,0.768750,0.970353,0.961163,0.929676,0.857866,0.854256,0.841589,4349.0,1.0,549.0,8.0,6939.0,53.0,2875.0,266.0
8,LAFTR4EqOpp,0.2,0.783710,0.933699,0.913047,0.845141,0.852155,0.843449,0.813267,4347.0,3.0,542.0,15.0,6854.0,138.0,2570.0,571.0
9,LAFTR4EqOpp,0.5,0.783710,0.933699,0.913047,0.845141,0.852155,0.843449,0.813267,4347.0,3.0,542.0,15.0,6854.0,138.0,2570.0,571.0


In [19]:
result_df.to_csv(f'adult-result/laftr-{epochs}.csv')